# Omission LFP Analysis

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
from collections import defaultdict
import re

In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats

In [3]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp
from spectral_connectivity import Multitaper, Connectivity

## Inputs & Data

Explanation of each input and where it comes from.

In [4]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

In [5]:
# variables for LFP extraction
FREQ_MIN=0.5
FREQ_MAX=300
FREQ=60
RESAMPLE_RATE=1000
TRIAL_DURATION=10

In [6]:
INPUT_VARIABLE = 1

TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 3
TIME_WINDOW_STEP = 1.5 

TRIAL_TIME_STAMP_DURATION = 1000*10

In [7]:
CHANNEL_MAPPING_DF = pd.read_excel("../../channel_mapping.xlsx")
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

TONE_TIMESTAMP_DF = pd.read_excel("../../rce_tone_timestamp.xlsx", index_col=0)
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs


In [8]:
all_session_dir = ['/scratch/back_up/reward_competition_extention/data/standard/2023_06_12/20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
'/scratch/back_up/reward_competition_extention/data/pilot/20221214_125409_om_and_comp_6_1_and_6_3.rec',
'/scratch/back_up/reward_competition_extention/data/pilot/20221215_145401_comp_amd_om_6_1_and_6_3.rec',
'/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec']

In [9]:
all_session_dir

['/scratch/back_up/reward_competition_extention/data/standard/2023_06_12/20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/pilot/20221214_125409_om_and_comp_6_1_and_6_3.rec',
 '/scratch/back_up/reward_competition_extention/data/pilot/20221215_145401_comp_amd_om_6_1_and_6_3.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions

- Ideally functions are defined here first and then data is processed using the functions
    - function names are short and in snake case all lowercase
    - a function name should be unique but does not have to describe the function
    - doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [10]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [11]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,9.0,31.0,30.0,29.0,28.0
7,2,1.4,NaN,15,14,13,12,15.0,31.0,30.0,29.0,28.0


### Getting the subject IDs from the file name

In [12]:
TONE_TIMESTAMP_DF = pd.read_pickle("./proc/merged_trials_and_video.pkl")


In [13]:
all_trials_df = TONE_TIMESTAMP_DF.dropna(subset="condition").reset_index(drop=True)

In [14]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,baseline_thorax_velocity,chunked_trial_thorax_velocity,chunked_baseline_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_trial_thorax_velocity,binned_baseline_thorax_velocity,baseline_and_trial_thorax_velocity
0,39560581,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,37057700,20230620_114347_standard_comp_to_omission_D4_s...,36988,2,1-1_t1b2L_box_2,...,"[1.4810881830626674, 1.6388684884656102, 1.952...","[1.7308062235215724, 4.236024740540817, 2.6081...","[5.748064615308862, 7.563465907100855, 1.28448...","[(37057700, 37077700), (37077700, 37097700), (...","[(36857700, 36877700), (36877700, 36897700), (...","[(1852885, 1853885), (1853885, 1854885), (1854...","[(1842885, 1843885), (1843885, 1844885), (1844...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[2, 2, 0, 1, 1, 0, 0, 0, 0, 0]","[1.4810881830626674, 1.6388684884656102, 1.952..."
1,40760598,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,38257717,20230620_114347_standard_comp_to_omission_D4_s...,38185,2,1-1_t1b2L_box_2,...,"[0.5708283919594127, 0.6083617293242782, 0.656...","[1.0209641619375505, 0.3775516993936581, 0.992...","[0.8839077228488512, 0.46772417569252217, 0.56...","[(38257717, 38277717), (38277717, 38297717), (...","[(38057717, 38077717), (38077717, 38097717), (...","[(1912885, 1913885), (1913885, 1914885), (1914...","[(1902885, 1903885), (1903885, 1904885), (1904...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.5708283919594127, 0.6083617293242782, 0.656..."
2,41860612,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,39357731,20230620_114347_standard_comp_to_omission_D4_s...,39283,2,1-1_t1b2L_box_2,...,"[1.242858806887267, 1.2597766713546783, 1.2554...","[0.8243034148456695, 1.0607847178968977, 1.454...","[1.612962216332033, 1.5749066904347877, 0.6808...","[(39357731, 39377731), (39377731, 39397731), (...","[(39157731, 39177731), (39177731, 39197731), (...","[(1967886, 1968886), (1968886, 1969886), (1969...","[(1957886, 1958886), (1958886, 1959886), (1959...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.242858806887267, 1.2597766713546783, 1.2554..."
3,43160631,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,40657750,20230620_114347_standard_comp_to_omission_D4_s...,40580,2,1-1_t1b2L_box_2,...,"[2.1606880861014384, 2.279700247812027, 2.3578...","[8.671122435857255, 0.7248409341210599, 0.8246...","[2.7278154096501344, 4.0641225383640505, 8.836...","[(40657750, 40677750), (40677750, 40697750), (...","[(40457750, 40477750), (40477750, 40497750), (...","[(2032887, 2033887), (2033887, 2034887), (2034...","[(2022887, 2023887), (2023887, 2024887), (2024...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 2, 2, 2, 1, 0, 2, 2, 2]","[2.1606880861014384, 2.279700247812027, 2.3578..."
4,44260639,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,41757758,20230620_114347_standard_comp_to_omission_D4_s...,41678,2,1-1_t1b2L_box_2,...,"[1.268819437205158, 1.1822592820891429, 1.1197...","[1.383219142457575, 1.973682437884362, 1.18558...","[2.256721229459507, 0.7367119396555359, 0.7671...","[(41757758, 41777758), (41777758, 41797758), (...","[(41557758, 41577758), (41577758, 41597758), (...","[(2087887, 2088887), (2088887, 2089887), (2089...","[(2077887, 2078887), (2078887, 2079887), (2079...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.268819437205158, 1.1822592820891429, 1.1197..."


- Original timestamps are based on ephys recordings at 20kHz. The LFP will be at 1kHz, so we will need to divide all the timestamps by 20

In [15]:
all_trials_df["resampled_index"] = all_trials_df["time_stamp_index"] // 20

In [16]:
all_trials_df["recording_dir"].unique()

array(['20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1'],
      dtype=object)

- Getting a list of all the subjects through the recording name

In [17]:
all_trials_df["all_subjects"] = all_trials_df["recording_dir"].apply(lambda x: ["{}.{}".format(tup[0],tup[1]) for tup in re.findall(r'(\d+)-(\d+)', x.replace("_", "-"))[1:]])

In [18]:
all_trials_df["all_subjects"].head()

0    [1.2, 1.1]
1    [1.2, 1.1]
2    [1.2, 1.1]
3    [1.2, 1.1]
4    [1.2, 1.1]
Name: all_subjects, dtype: object

- Getting the current subject of the recording through the ending of the recording name file

In [19]:
all_trials_df["subject_info"].head()

0    1-1_t1b2L_box_2
1    1-1_t1b2L_box_2
2    1-1_t1b2L_box_2
3    1-1_t1b2L_box_2
4    1-1_t1b2L_box_2
Name: subject_info, dtype: object

In [20]:
all_trials_df["current_subject"] = all_trials_df["subject_info"].apply(lambda x: ".".join(x.replace("-","_").split("_")[:2]))

In [21]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,baseline_thorax_velocity,chunked_trial_thorax_velocity,chunked_baseline_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_trial_thorax_velocity,binned_baseline_thorax_velocity,baseline_and_trial_thorax_velocity
0,39560581,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,37057700,20230620_114347_standard_comp_to_omission_D4_s...,36988,2,1-1_t1b2L_box_2,...,"[1.4810881830626674, 1.6388684884656102, 1.952...","[1.7308062235215724, 4.236024740540817, 2.6081...","[5.748064615308862, 7.563465907100855, 1.28448...","[(37057700, 37077700), (37077700, 37097700), (...","[(36857700, 36877700), (36877700, 36897700), (...","[(1852885, 1853885), (1853885, 1854885), (1854...","[(1842885, 1843885), (1843885, 1844885), (1844...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[2, 2, 0, 1, 1, 0, 0, 0, 0, 0]","[1.4810881830626674, 1.6388684884656102, 1.952..."
1,40760598,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,38257717,20230620_114347_standard_comp_to_omission_D4_s...,38185,2,1-1_t1b2L_box_2,...,"[0.5708283919594127, 0.6083617293242782, 0.656...","[1.0209641619375505, 0.3775516993936581, 0.992...","[0.8839077228488512, 0.46772417569252217, 0.56...","[(38257717, 38277717), (38277717, 38297717), (...","[(38057717, 38077717), (38077717, 38097717), (...","[(1912885, 1913885), (1913885, 1914885), (1914...","[(1902885, 1903885), (1903885, 1904885), (1904...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.5708283919594127, 0.6083617293242782, 0.656..."
2,41860612,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,39357731,20230620_114347_standard_comp_to_omission_D4_s...,39283,2,1-1_t1b2L_box_2,...,"[1.242858806887267, 1.2597766713546783, 1.2554...","[0.8243034148456695, 1.0607847178968977, 1.454...","[1.612962216332033, 1.5749066904347877, 0.6808...","[(39357731, 39377731), (39377731, 39397731), (...","[(39157731, 39177731), (39177731, 39197731), (...","[(1967886, 1968886), (1968886, 1969886), (1969...","[(1957886, 1958886), (1958886, 1959886), (1959...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.242858806887267, 1.2597766713546783, 1.2554..."
3,43160631,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,40657750,20230620_114347_standard_comp_to_omission_D4_s...,40580,2,1-1_t1b2L_box_2,...,"[2.1606880861014384, 2.279700247812027, 2.3578...","[8.671122435857255, 0.7248409341210599, 0.8246...","[2.7278154096501344, 4.0641225383640505, 8.836...","[(40657750, 40677750), (40677750, 40697750), (...","[(40457750, 40477750), (40477750, 40497750), (...","[(2032887, 2033887), (2033887, 2034887), (2034...","[(2022887, 2023887), (2023887, 2024887), (2024...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 2, 2, 2, 1, 0, 2, 2, 2]","[2.1606880861014384, 2.279700247812027, 2.3578..."
4,44260639,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,41757758,20230620_114347_standard_comp_to_omission_D4_s...,41678,2,1-1_t1b2L_box_2,...,"[1.268819437205158, 1.1822592820891429, 1.1197...","[1.383219142457575, 1.973682437884362, 1.18558...","[2.256721229459507, 0.7367119396555359, 0.7671...","[(41757758, 41777758), (41777758, 41797758), (...","[(41557758, 41577758), (41577758, 41597758), (...","[(2087887, 2088887), (2088887, 2089887), (2089...","[(2077887, 2078887), (2078887, 2079887), (2079...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.268819437205158, 1.1822592820891429, 1.1197..."


- Labeling the trial as a winner or loser if the winner matches the subject id or not

In [22]:
all_trials_df["trial_outcome"] = all_trials_df.apply(
    lambda x: "win" if str(x["condition"]).strip() == str(x["current_subject"]) 
             else ("lose" if str(x["condition"]) in x["all_subjects"] 
                   else x["condition"]), axis=1)

In [23]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,baseline_thorax_velocity,chunked_trial_thorax_velocity,chunked_baseline_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_trial_thorax_velocity,binned_baseline_thorax_velocity,baseline_and_trial_thorax_velocity
0,39560581,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,37057700,20230620_114347_standard_comp_to_omission_D4_s...,36988,2,1-1_t1b2L_box_2,...,"[1.4810881830626674, 1.6388684884656102, 1.952...","[1.7308062235215724, 4.236024740540817, 2.6081...","[5.748064615308862, 7.563465907100855, 1.28448...","[(37057700, 37077700), (37077700, 37097700), (...","[(36857700, 36877700), (36877700, 36897700), (...","[(1852885, 1853885), (1853885, 1854885), (1854...","[(1842885, 1843885), (1843885, 1844885), (1844...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[2, 2, 0, 1, 1, 0, 0, 0, 0, 0]","[1.4810881830626674, 1.6388684884656102, 1.952..."
1,40760598,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,38257717,20230620_114347_standard_comp_to_omission_D4_s...,38185,2,1-1_t1b2L_box_2,...,"[0.5708283919594127, 0.6083617293242782, 0.656...","[1.0209641619375505, 0.3775516993936581, 0.992...","[0.8839077228488512, 0.46772417569252217, 0.56...","[(38257717, 38277717), (38277717, 38297717), (...","[(38057717, 38077717), (38077717, 38097717), (...","[(1912885, 1913885), (1913885, 1914885), (1914...","[(1902885, 1903885), (1903885, 1904885), (1904...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.5708283919594127, 0.6083617293242782, 0.656..."
2,41860612,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,39357731,20230620_114347_standard_comp_to_omission_D4_s...,39283,2,1-1_t1b2L_box_2,...,"[1.242858806887267, 1.2597766713546783, 1.2554...","[0.8243034148456695, 1.0607847178968977, 1.454...","[1.612962216332033, 1.5749066904347877, 0.6808...","[(39357731, 39377731), (39377731, 39397731), (...","[(39157731, 39177731), (39177731, 39197731), (...","[(1967886, 1968886), (1968886, 1969886), (1969...","[(1957886, 1958886), (1958886, 1959886), (1959...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.242858806887267, 1.2597766713546783, 1.2554..."
3,43160631,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,40657750,20230620_114347_standard_comp_to_omission_D4_s...,40580,2,1-1_t1b2L_box_2,...,"[2.1606880861014384, 2.279700247812027, 2.3578...","[8.671122435857255, 0.7248409341210599, 0.8246...","[2.7278154096501344, 4.0641225383640505, 8.836...","[(40657750, 40677750), (40677750, 40697750), (...","[(40457750, 40477750), (40477750, 40497750), (...","[(2032887, 2033887), (2033887, 2034887), (2034...","[(2022887, 2023887), (2023887, 2024887), (2024...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 2, 2, 2, 1, 0, 2, 2, 2]","[2.1606880861014384, 2.279700247812027, 2.3578..."
4,44260639,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,41757758,20230620_114347_standard_comp_to_omission_D4_s...,41678,2,1-1_t1b2L_box_2,...,"[1.268819437205158, 1.1822592820891429, 1.1197...","[1.383219142457575, 1.973682437884362, 1.18558...","[2.256721229459507, 0.7367119396555359, 0.7671...","[(41757758, 41777758), (41777758, 41797758), (...","[(41557758, 41577758), (41577758, 41597758), (...","[(2087887, 2088887), (2088887, 2089887), (2089...","[(2077887, 2078887), (2078887, 2079887), (2079...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.268819437205158, 1.1822592820891429, 1.1197..."


In [24]:
all_trials_df

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,baseline_thorax_velocity,chunked_trial_thorax_velocity,chunked_baseline_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_trial_thorax_velocity,binned_baseline_thorax_velocity,baseline_and_trial_thorax_velocity
0,39560581,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,37057700,20230620_114347_standard_comp_to_omission_D4_s...,36988,2,1-1_t1b2L_box_2,...,"[1.4810881830626674, 1.6388684884656102, 1.952...","[1.7308062235215724, 4.236024740540817, 2.6081...","[5.748064615308862, 7.563465907100855, 1.28448...","[(37057700, 37077700), (37077700, 37097700), (...","[(36857700, 36877700), (36877700, 36897700), (...","[(1852885, 1853885), (1853885, 1854885), (1854...","[(1842885, 1843885), (1843885, 1844885), (1844...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[2, 2, 0, 1, 1, 0, 0, 0, 0, 0]","[1.4810881830626674, 1.6388684884656102, 1.952..."
1,40760598,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,38257717,20230620_114347_standard_comp_to_omission_D4_s...,38185,2,1-1_t1b2L_box_2,...,"[0.5708283919594127, 0.6083617293242782, 0.656...","[1.0209641619375505, 0.3775516993936581, 0.992...","[0.8839077228488512, 0.46772417569252217, 0.56...","[(38257717, 38277717), (38277717, 38297717), (...","[(38057717, 38077717), (38077717, 38097717), (...","[(1912885, 1913885), (1913885, 1914885), (1914...","[(1902885, 1903885), (1903885, 1904885), (1904...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.5708283919594127, 0.6083617293242782, 0.656..."
2,41860612,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,39357731,20230620_114347_standard_comp_to_omission_D4_s...,39283,2,1-1_t1b2L_box_2,...,"[1.242858806887267, 1.2597766713546783, 1.2554...","[0.8243034148456695, 1.0607847178968977, 1.454...","[1.612962216332033, 1.5749066904347877, 0.6808...","[(39357731, 39377731), (39377731, 39397731), (...","[(39157731, 39177731), (39177731, 39197731), (...","[(1967886, 1968886), (1968886, 1969886), (1969...","[(1957886, 1958886), (1958886, 1959886), (1959...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.242858806887267, 1.2597766713546783, 1.2554..."
3,43160631,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,40657750,20230620_114347_standard_comp_to_omission_D4_s...,40580,2,1-1_t1b2L_box_2,...,"[2.1606880861014384, 2.279700247812027, 2.3578...","[8.671122435857255, 0.7248409341210599, 0.8246...","[2.7278154096501344, 4.0641225383640505, 8.836...","[(40657750, 40677750), (40677750, 40697750), (...","[(40457750, 40477750), (40477750, 40497750), (...","[(2032887, 2033887), (2033887, 2034887), (2034...","[(2022887, 2023887), (2023887, 2024887), (2024...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 2, 2, 2, 1, 0, 2, 2, 2]","[2.1606880861014384, 2.279700247812027, 2.3578..."
4,44260639,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,41757758,20230620_114347_standard_comp_to_omission_D4_s...,41678,2,1-1_t1b2L_box_2,...,"[1.268819437205158, 1.1822592820891429, 1.1197...","[1.383219142457575, 1.973682437884362, 1.18558...","[2.256721229459507, 0.7367119396555359, 0.7671...","[(41757758, 41777758), (41777758, 41797758), (...","[(41557758, 41577758), (41577758, 41597758), (...","[(2087887, 2088887), (2088887, 2089887), (2089...","[(2077887, 2078887), (2078887, 2079887), (2079...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.268819437205158, 1.1822592820891429, 1.1197..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,59034912,1,20230618_

### Extracting the LFP

In [25]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in all_session_dir:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, "*.rec")):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id="ECU")
            current_recording = se.read_spikegadgets(recording_path, stream_id="trodes")
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=0.5, freq_max=300)
            current_recording = sp.notch_filter(current_recording, freq=60)
            current_recording = sp.resample(current_recording, resample_rate=1000)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except:
            pass



20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged
20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged
20221214_125409_om_and_comp_6_1_top_1_base_2_vs_6_3
20221215_145401_comp_amd_om_6_1_top_4_base_3
20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged
20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged


- Filtering for all trials that have labels

In [26]:
all_trials_df = all_trials_df[all_trials_df["recording_file"].isin(recording_name_to_all_ch_lfp.keys())].reset_index(drop=True)

In [27]:
baseline_df = all_trials_df.copy()

In [28]:
baseline_df["trial_outcome"] = baseline_df["trial_outcome"] + "_baseline"

In [29]:
baseline_df["resampled_index"] = baseline_df["resampled_index"] - (RESAMPLE_RATE * TRIAL_DURATION)

In [30]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,baseline_thorax_velocity,chunked_trial_thorax_velocity,chunked_baseline_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_trial_thorax_velocity,binned_baseline_thorax_velocity,baseline_and_trial_thorax_velocity
0,39560581,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,37057700,20230620_114347_standard_comp_to_omission_D4_s...,36988,2,1-1_t1b2L_box_2,...,"[1.4810881830626674, 1.6388684884656102, 1.952...","[1.7308062235215724, 4.236024740540817, 2.6081...","[5.748064615308862, 7.563465907100855, 1.28448...","[(37057700, 37077700), (37077700, 37097700), (...","[(36857700, 36877700), (36877700, 36897700), (...","[(1852885, 1853885), (1853885, 1854885), (1854...","[(1842885, 1843885), (1843885, 1844885), (1844...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[2, 2, 0, 1, 1, 0, 0, 0, 0, 0]","[1.4810881830626674, 1.6388684884656102, 1.952..."
1,40760598,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,38257717,20230620_114347_standard_comp_to_omission_D4_s...,38185,2,1-1_t1b2L_box_2,...,"[0.5708283919594127, 0.6083617293242782, 0.656...","[1.0209641619375505, 0.3775516993936581, 0.992...","[0.8839077228488512, 0.46772417569252217, 0.56...","[(38257717, 38277717), (38277717, 38297717), (...","[(38057717, 38077717), (38077717, 38097717), (...","[(1912885, 1913885), (1913885, 1914885), (1914...","[(1902885, 1903885), (1903885, 1904885), (1904...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.5708283919594127, 0.6083617293242782, 0.656..."
2,41860612,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,39357731,20230620_114347_standard_comp_to_omission_D4_s...,39283,2,1-1_t1b2L_box_2,...,"[1.242858806887267, 1.2597766713546783, 1.2554...","[0.8243034148456695, 1.0607847178968977, 1.454...","[1.612962216332033, 1.5749066904347877, 0.6808...","[(39357731, 39377731), (39377731, 39397731), (...","[(39157731, 39177731), (39177731, 39197731), (...","[(1967886, 1968886), (1968886, 1969886), (1969...","[(1957886, 1958886), (1958886, 1959886), (1959...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.242858806887267, 1.2597766713546783, 1.2554..."
3,43160631,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,40657750,20230620_114347_standard_comp_to_omission_D4_s...,40580,2,1-1_t1b2L_box_2,...,"[2.1606880861014384, 2.279700247812027, 2.3578...","[8.671122435857255, 0.7248409341210599, 0.8246...","[2.7278154096501344, 4.0641225383640505, 8.836...","[(40657750, 40677750), (40677750, 40697750), (...","[(40457750, 40477750), (40477750, 40497750), (...","[(2032887, 2033887), (2033887, 2034887), (2034...","[(2022887, 2023887), (2023887, 2024887), (2024...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 2, 2, 2, 1, 0, 2, 2, 2]","[2.1606880861014384, 2.279700247812027, 2.3578..."
4,44260639,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,41757758,20230620_114347_standard_comp_to_omission_D4_s...,41678,2,1-1_t1b2L_box_2,...,"[1.268819437205158, 1.1822592820891429, 1.1197...","[1.383219142457575, 1.973682437884362, 1.18558...","[2.256721229459507, 0.7367119396555359, 0.7671...","[(41757758, 41777758), (41777758, 41797758), (...","[(41557758, 41577758), (41577758, 41597758), (...","[(2087887, 2088887), (2088887, 2089887), (2089...","[(2077887, 2078887), (2078887, 2079887), (2079...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.268819437205158, 1.1822592820891429, 1.1197..."


In [31]:
baseline_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,baseline_thorax_velocity,chunked_trial_thorax_velocity,chunked_baseline_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_trial_thorax_velocity,binned_baseline_thorax_velocity,baseline_and_trial_thorax_velocity
0,39560581,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,37057700,20230620_114347_standard_comp_to_omission_D4_s...,36988,2,1-1_t1b2L_box_2,...,"[1.4810881830626674, 1.6388684884656102, 1.952...","[1.7308062235215724, 4.236024740540817, 2.6081...","[5.748064615308862, 7.563465907100855, 1.28448...","[(37057700, 37077700), (37077700, 37097700), (...","[(36857700, 36877700), (36877700, 36897700), (...","[(1852885, 1853885), (1853885, 1854885), (1854...","[(1842885, 1843885), (1843885, 1844885), (1844...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[2, 2, 0, 1, 1, 0, 0, 0, 0, 0]","[1.4810881830626674, 1.6388684884656102, 1.952..."
1,40760598,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,38257717,20230620_114347_standard_comp_to_omission_D4_s...,38185,2,1-1_t1b2L_box_2,...,"[0.5708283919594127, 0.6083617293242782, 0.656...","[1.0209641619375505, 0.3775516993936581, 0.992...","[0.8839077228488512, 0.46772417569252217, 0.56...","[(38257717, 38277717), (38277717, 38297717), (...","[(38057717, 38077717), (38077717, 38097717), (...","[(1912885, 1913885), (1913885, 1914885), (1914...","[(1902885, 1903885), (1903885, 1904885), (1904...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.5708283919594127, 0.6083617293242782, 0.656..."
2,41860612,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,39357731,20230620_114347_standard_comp_to_omission_D4_s...,39283,2,1-1_t1b2L_box_2,...,"[1.242858806887267, 1.2597766713546783, 1.2554...","[0.8243034148456695, 1.0607847178968977, 1.454...","[1.612962216332033, 1.5749066904347877, 0.6808...","[(39357731, 39377731), (39377731, 39397731), (...","[(39157731, 39177731), (39177731, 39197731), (...","[(1967886, 1968886), (1968886, 1969886), (1969...","[(1957886, 1958886), (1958886, 1959886), (1959...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.242858806887267, 1.2597766713546783, 1.2554..."
3,43160631,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,40657750,20230620_114347_standard_comp_to_omission_D4_s...,40580,2,1-1_t1b2L_box_2,...,"[2.1606880861014384, 2.279700247812027, 2.3578...","[8.671122435857255, 0.7248409341210599, 0.8246...","[2.7278154096501344, 4.0641225383640505, 8.836...","[(40657750, 40677750), (40677750, 40697750), (...","[(40457750, 40477750), (40477750, 40497750), (...","[(2032887, 2033887), (2033887, 2034887), (2034...","[(2022887, 2023887), (2023887, 2024887), (2024...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 2, 2, 2, 1, 0, 2, 2, 2]","[2.1606880861014384, 2.279700247812027, 2.3578..."
4,44260639,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,41757758,20230620_114347_standard_comp_to_omission_D4_s...,41678,2,1-1_t1b2L_box_2,...,"[1.268819437205158, 1.1822592820891429, 1.1197...","[1.383219142457575, 1.973682437884362, 1.18558...","[2.256721229459507, 0.7367119396555359, 0.7671...","[(41757758, 41777758), (41777758, 41797758), (...","[(41557758, 41577758), (41577758, 41597758), (...","[(2087887, 2088887), (2088887, 2089887), (2089...","[(2077887, 2078887), (2078887, 2079887), (2079...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.268819437205158, 1.1822592820891429, 1.1197..."


In [32]:
all_trials_df = pd.concat([all_trials_df, baseline_df]).reset_index(drop=True).sort_values(["recording_file", "time"])

In [33]:
all_trials_df["trial_outcome"].unique()

array(['lose', 'lose_baseline', 'win', 'win_baseline', 'rewarded',
       'rewarded_baseline', 'omission', 'omission_baseline'], dtype=object)

In [34]:
recording_name_to_all_ch_lfp.keys()

dict_keys(['20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged', '20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged', '20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged', '20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged', '20221214_125409_om_and_comp_6_1_top_1_base_2_vs_6_3', '20221215_145401_comp_amd_om_6_1_top_4_base_3', '20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged', '20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged'])

In [35]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,9.0,31.0,30.0,29.0,28.0
7,2,1.4,NaN,15,14,13,12,15.0,31.0,30.0,29.0,28.0


## Power Calculation

In [36]:
all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,baseline_thorax_velocity,chunked_trial_thorax_velocity,chunked_baseline_thorax_velocity,trial_chunked_ephys_timestamp,baseline_chunked_ephys_timestamp,trial_chunked_lfp_index,baseline_chunked_lfp_index,binned_trial_thorax_velocity,binned_baseline_thorax_velocity,baseline_and_trial_thorax_velocity
74,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,"[0.33367330724769034, 0.32906494406630954, 0.3...","[9.873337358573975, 7.055864053204467, 0.39102...","[0.43638923644394456, 0.5639966810493696, 0.57...","[(1099250, 1119250), (1119250, 1139250), (1139...","[(899250, 919250), (919250, 939250), (939250, ...","[(54962, 55962), (55962, 56962), (56962, 57962...","[(44962, 45962), (45962, 46962), (46962, 47962...","[2, 2, 0, 0, 1, 2, 2, 1, 2, 2]","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2]","[0.33367330724769034, 0.32906494406630954, 0.3..."
226,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,"[0.33367330724769034, 0.32906494406630954, 0.3...","[9.873337358573975, 7.055864053204467, 0.39102...","[0.43638923644394456, 0.5639966810493696, 0.57...","[(1099250, 1119250), (1119250, 1139250), (1139...","[(899250, 919250), (919250, 939250), (939250, ...","[(54962, 55962), (55962, 56962), (56962, 57962...","[(44962, 45962), (45962, 46962), (46962, 47962...","[2, 2, 0, 0, 1, 2, 2, 1, 2, 2]","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2]","[0.33367330724769034, 0.32906494406630954, 0.3..."
75,4334936,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,3499256,20230618_100636_standard_comp_to_omission_D2_s...,3492,1,1_1_t1b2L_box2,...,"[0.7987128002646031, 0.8394469016952293, 0.845...","[6.20912009533163, 3.1460093621391327, 1.76285...","[0.8478014795615629, 1.225847562493769, 1.2524...","[(3499256, 3519256), (3519256, 3539256), (3539...","[(3299256, 3319256), (3319256, 3339256), (3339...","[(174962, 175962), (175962, 176962), (176962, ...","[(164962, 165962), (165962, 166962), (166962, ...","[2, 1, 0, 1, 2, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 2, 1, 2, 2]","[0.7987128002646031, 0.8394469016952293, 0.845..."
227,4334936,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,3499256,20230618_100636_standard_comp_to_omission_D2_s...,3492,1,1_1_t1b2L_box2,...,"[0.7987128002646031, 0.8394469016952293, 0.845...","[6.20912009533163, 3.1460093621391327, 1.76285...","[0.8478014795615629, 1.225847562493769, 1.2524...","[(3499256, 3519256), (3519256, 3539256), (3539...","[(3299256, 3319256), (3319256, 3339256), (3339...","[(174962, 175962), (175962, 176962), (176962, ...","[(164962, 165962), (165962, 166962), (166962, ...","[2, 1, 0, 1, 2, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 2, 1, 2, 2]","[0.7987128002646031, 0.8394469016952293, 0.845..."
76,6634931,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,5799251,20230618_100636_standard_comp_to_omission_D2_s...,5788,1,1_1_t1b2L_box2,...,"[1.1035883660509216, 1.103912980719602, 1.1008...","[0.4663992604047454, 1.6244034133379446, 4.535...","[0.6305645270264292, 0.7913440867553568, 4.059...","[(5799251, 5819251), (5819251, 5839251), (5839...","[(5599251, 5619251), (5619251, 5639251), (5639...","[(289962, 290962), (290962, 291962), (291962, ...","[(279962, 280962), (280962, 281962), (281962, ...","[0, 0, 1, 0, 0, 1, 2, 2, 0, 0]","[0, 0, 1, 0, 1, 0, 1, 1, 1, 0]","[1.1035883660509216, 1.103912980719602, 1.1008..."


- Adding all the brain region to ch information

In [37]:
channel_map_and_all_trials_df = all_trials_df.merge(CHANNEL_MAPPING_DF, left_on="current_subject", right_on="Subject", how="left")

- Linking up all LFP calculations with all the trials

In [38]:
channel_map_and_all_trials_df["all_ch_lfp"] = channel_map_and_all_trials_df["recording_file"].map(recording_name_to_all_ch_lfp)

- Creating a new row for each brain region

In [39]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if "spike_interface" in col]

In [40]:
id_cols = [col for col in channel_map_and_all_trials_df.columns if col not in brain_region_col]

In [41]:
melted_channel_map_and_all_trials_df = channel_map_and_all_trials_df.melt(id_vars=id_cols ,value_vars=brain_region_col, var_name='brain_region', value_name='channel')


In [42]:
melted_channel_map_and_all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,all_ch_lfp,brain_region,channel
0,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
1,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
2,4334936,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,3499256,20230618_100636_standard_comp_to_omission_D2_s...,3492,1,1_1_t1b2L_box2,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
3,4334936,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,3499256,20230618_100636_standard_comp_to_omission_D2_s...,3492,1,1_1_t1b2L_box2,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0
4,6634931,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,5799251,20230618_100636_standard_comp_to_omission_D2_s...,5788,1,1_1_t1b2L_box2,...,2,1.1,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5.0


In [43]:
melted_channel_map_and_all_trials_df.tail()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,all_ch_lfp,brain_region,channel
1515,66660918,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,64158037,20230620_114347_standard_comp_to_omission_D4_s...,64089,1,1-2_t3b3L_box_1,...,2,1.2,NaN,31,30,29,28,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
1516,68560944,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,66058063,20230620_114347_standard_comp_to_omission_D4_s...,65986,1,1-2_t3b3L_box_1,...,2,1.2,NaN,31,30,29,28,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
1517,68560944,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,66058063,20230620_114347_standard_comp_to_omission_D4_s...,65986,1,1-2_t3b3L_box_1,...,2,1.2,NaN,31,30,29,28,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
1518,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,2,1.2,NaN,31,30,29,28,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0
1519,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,2,1.2,NaN,31,30,29,28,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28.0


- Getting the traces for each trial for each brain region

In [44]:
melted_channel_map_and_all_trials_df["channel"] = melted_channel_map_and_all_trials_df["channel"].astype(int).astype(str)
melted_channel_map_and_all_trials_df["resampled_index"] = melted_channel_map_and_all_trials_df["resampled_index"].astype(int)

In [71]:
melted_channel_map_and_all_trials_df["trial_chunked_lfp_index"].iloc[0]

[(54962, 55962),
 (55962, 56962),
 (56962, 57962),
 (57962, 58962),
 (58962, 59962),
 (59962, 60962),
 (60962, 61962),
 (61962, 62962),
 (62962, 63962),
 (63962, 64962)]

In [72]:
melted_channel_map_and_all_trials_df["binned_trial_thorax_velocity"].iloc[0]

[2, 2, 0, 0, 1, 2, 2, 1, 2, 2]

In [45]:
melted_channel_map_and_all_trials_df["chunked_trial_trace"] = melted_channel_map_and_all_trials_df.apply(lambda x: 
[x["all_ch_lfp"].get_traces(channel_ids=[x["channel"]], start_frame=chunk_index[0], end_frame=chunk_index[1]).T[0] for chunk_index in x["trial_chunked_lfp_index"]], axis=1)

In [46]:
melted_channel_map_and_all_trials_df["chunked_baseline_trace"] = melted_channel_map_and_all_trials_df.apply(lambda x: 
[x["all_ch_lfp"].get_traces(channel_ids=[x["channel"]], start_frame=chunk_index[0], end_frame=chunk_index[1]).T[0] for chunk_index in x["baseline_chunked_lfp_index"]], axis=1)

In [47]:
len(melted_channel_map_and_all_trials_df["chunked_baseline_trace"].iloc[0])

10

In [48]:
melted_channel_map_and_all_trials_df["binned_baseline_thorax_velocity"]

0       [0, 0, 0, 0, 0, 1, 2, 2, 2, 2]
1       [0, 0, 0, 0, 0, 1, 2, 2, 2, 2]
2       [0, 0, 0, 0, 0, 0, 2, 1, 2, 2]
3       [0, 0, 0, 0, 0, 0, 2, 1, 2, 2]
4       [0, 0, 1, 0, 1, 0, 1, 1, 1, 0]
                     ...              
1515    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1516    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1517    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1518    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
1519    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Name: binned_baseline_thorax_velocity, Length: 1520, dtype: object

In [49]:
melted_channel_map_and_all_trials_df["chunked_baseline_trace"]

0       [[1.2846744, 1.113752, 0.7553665, 0.60466075, ...
1       [[1.2846744, 1.113752, 0.7553665, 0.60466075, ...
2       [[-0.4557929, -0.48152316, -0.32897955, -0.194...
3       [[-0.4557929, -0.48152316, -0.32897955, -0.194...
4       [[0.077190734, -0.19849047, -0.40065667, -0.56...
                              ...                        
1515    [[0.027177656, -0.38789928, -0.32860255, -0.13...
1516    [[0.9363938, 1.0475751, 1.2649963, 1.3069981, ...
1517    [[0.9363938, 1.0475751, 1.2649963, 1.3069981, ...
1518    [[1.2155824, 0.6102619, -0.022236263, -0.38048...
1519    [[1.2155824, 0.6102619, -0.022236263, -0.38048...
Name: chunked_baseline_trace, Length: 1520, dtype: object

In [50]:
exploded_channel_map_and_all_trials_df = melted_channel_map_and_all_trials_df.explode(["chunked_baseline_trace", "chunked_trial_trace", "binned_baseline_thorax_velocity", "binned_trial_thorax_velocity", "trial_chunked_lfp_index", "baseline_chunked_lfp_index"])

In [51]:
exploded_channel_map_and_all_trials_df

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,all_ch_lfp,brain_region,channel,chunked_trial_trace,chunked_baseline_trace
0,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5,"[0.36573705, 0.3547098, 0.32162806, 0.25546458...","[1.2846744, 1.113752, 0.7553665, 0.60466075, 0..."
0,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5,"[-1.1358066, -1.0861839, -0.9079101, -0.566065...","[1.4776512, 1.4813269, 1.3287834, 1.1174278, 1..."
0,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5,"[1.1229414, 1.4280286, 1.304891, 1.3398106, 1....","[-0.88034195, -0.4502793, 0.033081744, -0.1249..."
0,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5,"[2.345128, 2.1172316, 1.8985245, 1.9610122, 2....","[0.093731605, -0.05513624, -0.24443734, -0.317..."
0,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,NaN,16,17,18,19,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_mPFC,5,"[1.1615368, 1.20197, 1.0898597, 0.70758176, 0....","[-0.40249455, -0.51092917, -0.46314442, -0.382..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,NaN,31,30,29,28,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28,"[-1.1488736, -0.8647436, -0.6843828, -0.723913...","[0.7239139, 1.0228682, 0.9709835, 0.71650183, ..."
1519,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,NaN,31,30,29,28,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28,"[-1.1068718, -0.6769707, -0.71650183, -0.95121...","[0.32860255, 0.34836814, 0.36072162, 0.4867271..."
1519,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,NaN,31,30,29,28,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28,"[-0.5336703, -0.39284065, -0.6769707, -0.69920...","[0.121064104, 0.42495972, 0.63249815, 0.736267..."
1519,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,NaN,31,30,29,28,ZScoreRecording: 32 channels - 1 segments - 1....,spike_interface_MD,28,"[-1.8653754, -1.6306593, -1.7863132, -1.761606...","[0.0024706959, 0.13588828, 0.14824176, -0.0914..."


In [52]:
exploded_channel_map_and_all_trials_df["len_chunked_trial_trace"] = exploded_channel_map_and_all_trials_df["chunked_trial_trace"].apply(lambda x: len(x))

exploded_channel_map_and_all_trials_df["len_chunked_baseline_trace"] = exploded_channel_map_and_all_trials_df["chunked_baseline_trace"].apply(lambda x: len(x))

In [53]:
exploded_channel_map_and_all_trials_df["len_chunked_trial_trace"].unique()

array([1000, 1001])

In [54]:
exploded_channel_map_and_all_trials_df["len_chunked_baseline_trace"].unique()

array([1000])

In [55]:
exploded_channel_map_and_all_trials_df["chunked_trial_trace"] = exploded_channel_map_and_all_trials_df["chunked_trial_trace"].apply(lambda x: x[:RESAMPLE_RATE])

In [56]:
exploded_channel_map_and_all_trials_df["chunked_baseline_trace"] = exploded_channel_map_and_all_trials_df["chunked_baseline_trace"].apply(lambda x: x[:RESAMPLE_RATE])

In [57]:
exploded_channel_map_and_all_trials_df["chunked_trial_trace"]

0       [0.36573705, 0.3547098, 0.32162806, 0.25546458...
0       [-1.1358066, -1.0861839, -0.9079101, -0.566065...
0       [1.1229414, 1.4280286, 1.304891, 1.3398106, 1....
0       [2.345128, 2.1172316, 1.8985245, 1.9610122, 2....
0       [1.1615368, 1.20197, 1.0898597, 0.70758176, 0....
                              ...                        
1519    [-1.1488736, -0.8647436, -0.6843828, -0.723913...
1519    [-1.1068718, -0.6769707, -0.71650183, -0.95121...
1519    [-0.5336703, -0.39284065, -0.6769707, -0.69920...
1519    [-1.8653754, -1.6306593, -1.7863132, -1.761606...
1519    [-2.5719945, -2.7202363, -2.5868187, -2.315042...
Name: chunked_trial_trace, Length: 15200, dtype: object

In [58]:
chunked_columns = ["chunked_trial_trace", "chunked_baseline_trace"]

In [59]:
id_vars = [col for col in exploded_channel_map_and_all_trials_df.columns if col not in chunked_columns]

In [60]:
exploded_channel_map_and_all_trials_df = exploded_channel_map_and_all_trials_df.melt(id_vars=id_vars, value_vars=chunked_columns, var_name='trial_or_baseline', value_name='chunked_trace')

- Calcuating the power at each frequency band

In [61]:
exploded_channel_map_and_all_trials_df["multitaper"] = exploded_channel_map_and_all_trials_df["chunked_trace"].apply(lambda x: Multitaper(time_series=x,                                                                                        sampling_frequency=RESAMPLE_RATE))

In [62]:
exploded_channel_map_and_all_trials_df["connectivity"] = exploded_channel_map_and_all_trials_df["multitaper"].apply(lambda x: Connectivity.from_multitaper(x))

In [63]:
exploded_channel_map_and_all_trials_df["frequencies"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.frequencies)

In [64]:
exploded_channel_map_and_all_trials_df["power"] = exploded_channel_map_and_all_trials_df["connectivity"].apply(lambda x: x.power().squeeze())

In [65]:
exploded_channel_map_and_all_trials_df.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,brain_region,channel,len_chunked_trial_trace,len_chunked_baseline_trace,trial_or_baseline,chunked_trace,multitaper,connectivity,frequencies,power
0,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,spike_interface_mPFC,5,1000,1000,chunked_trial_trace,"[0.36573705, 0.3547098, 0.32162806, 0.25546458...","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0015864864245552536, 0.0043586493305438805,..."
1,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,spike_interface_mPFC,5,1000,1000,chunked_trial_trace,"[-1.1358066, -1.0861839, -0.9079101, -0.566065...","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.01017303674886765, 0.00949212589751741, 0.0..."
2,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,spike_interface_mPFC,5,1000,1000,chunked_trial_trace,"[1.1229414, 1.4280286, 1.304891, 1.3398106, 1....","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.07411999886535062, 0.07784380021192404, 0.1..."
3,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,spike_interface_mPFC,5,1000,1000,chunked_trial_trace,"[2.345128, 2.1172316, 1.8985245, 1.9610122, 2....","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.025733516240737022, 0.03179149934755682, 0...."
4,1934930,1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,dio_ECU_Din1,1099250,20230618_100636_standard_comp_to_omission_D2_s...,1097,1,1_1_t1b2L_box2,...,spike_interface_mPFC,5,1000,1000,chunked_trial_trace,"[1.1615368, 1.20197, 1.0898597, 0.70758176, 0....","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.014010667675424427, 0.024242772712469857, 0..."


In [69]:
exploded_channel_map_and_all_trials_df.tail()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,...,brain_region,channel,len_chunked_trial_trace,len_chunked_baseline_trace,trial_or_baseline,chunked_trace,multitaper,connectivity,frequencies,power
30395,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,spike_interface_MD,28,1000,1000,chunked_baseline_trace,"[0.7239139, 1.0228682, 0.9709835, 0.71650183, ...","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.020535680633812094, 0.019099368749882335, 0..."
30396,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,spike_interface_MD,28,1000,1000,chunked_baseline_trace,"[0.32860255, 0.34836814, 0.36072162, 0.4867271...","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.012548989130217209, 0.018010763844255373, 0..."
30397,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,spike_interface_MD,28,1000,1000,chunked_baseline_trace,"[0.121064104, 0.42495972, 0.63249815, 0.736267...","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.017297100287311486, 0.028694702677692425, 0..."
30398,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,spike_interface_MD,28,1000,1000,chunked_baseline_trace,"[0.0024706959, 0.13588828, 0.14824176, -0.0914...","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.032220251247796594, 0.025235414329778428, 0..."
30399,69560954,1,20230620_114347_standard_comp_to_omission_D4_s...,20230620_114347_standard_comp_to_omission_D4_s...,dio_ECU_Din1,67058073,20230620_114347_standard_comp_to_omission_D4_s...,66984,1,1-2_t3b3L_box_1,...,spike_interface_MD,28,1000,1000,chunked_baseline_trace,"[2.9697766, 2.7770622, 2.7745917, 2.80424, 2.6...","Multitaper(sampling_frequency=1000, time_halfb...",<spectral_connectivity.connectivity.Connectivi...,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.05760564326256991, 0.046412961093502125, 0...."


In [68]:
exploded_channel_map_and_all_trials_df["binned_thorax_velocity"]

0        2
1        2
2        0
3        0
4        1
        ..
30395    0
30396    0
30397    0
30398    0
30399    0
Name: binned_trial_thorax_velocity, Length: 30400, dtype: object

In [66]:
raise ValueError()

ValueError: 

In [ ]:
grouped_all_trials_df = exploded_channel_map_and_all_trials_df.groupby(['brain_region', 'trial_outcome', 'recording_file', 'current_subject']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df = exploded_channel_map_and_all_trials_df.groupby(['brain_region', 'trial_outcome', 'current_subject']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
color_list = ["b", "g", "r", "m", "y", "pink", "purple"]

In [ ]:
id_to_color = { id: color_list[index] for index, id in enumerate(sorted(grouped_all_trials_df["current_subject"].unique()))}

In [ ]:
id_to_color

In [ ]:
outcome_to_style = {'lose': "solid", 'omission': "dotted", 'rewarded': "dotted", 'win': "solid"}

In [ ]:
outcome_to_style = {'lose': "-", 'omission': "--", 'rewarded': "-", 'win': "--"}

In [ ]:
outcome_to_style = {'lose': "solid", 'omission': "dashed", 'rewarded': "dashed", 'win': "solid"}

In [ ]:
grouped_all_trials_df["trial_outcome"].unique()

In [ ]:
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(0, 12)
    plt.ylim(0, 0.15)
    # plt.yscale("log")

    for outcome in ["lose", "omission"]:
        outcome_df = region_df[region_df["trial_outcome"] == outcome]
        for index, row in outcome_df.iterrows():
            try:
                # print(row["mean_power"])
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], label="{} {}".format(outcome, row["current_subject"]), color=id_to_color[row["current_subject"]], linestyle=outcome_to_style[outcome], linewidth=3)
                # ax.lines[0].set_linestyle(outcome_to_style[outcome])
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2, color=id_to_color[row["current_subject"]])
            except Exception as e: 
                print(e)
                continue
    ax.legend(bbox_to_anchor=(1.3, 0.9), loc='upper right')
    plt.tight_layout()
    plt.savefig("./proc/z_scored_lose_and_omission_for_{}.png".format(region))
    # # Removing duplicate labels in the legend
    # handles, labels = plt.gca().get_legend_handles_labels()
    # unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    # plt.legend(*zip(*unique))

In [ ]:
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(0, 12)
    plt.ylim(0, 0.15)
    # plt.yscale("log")
    for outcome in ["rewarded", "win"]:
        outcome_df = region_df[region_df["trial_outcome"] == outcome]
        for index, row in outcome_df.iterrows():
            ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], label="{} {}".format(outcome, row["current_subject"]), color=id_to_color[row["current_subject"]], linestyle=outcome_to_style[outcome], linewidth=3)
            # ax.lines[0].set_linestyle(outcome_to_style[outcome])
            plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2, color=id_to_color[row["current_subject"]])
    ax.legend(bbox_to_anchor=(1.3, 0.9), loc='upper right')
    plt.tight_layout()

    plt.savefig("./proc/z_scored_win_and_rewarded_for_{}.png".format(region))

    # # Removing duplicate labels in the legend
    # handles, labels = plt.gca().get_legend_handles_labels()
    # unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    # plt.legend(*zip(*unique))

## Drawing plots with baseline

1 plot for each subject 

In [ ]:
outcome_to_style = {k: ("dashed" if "baseline" in k else "solid") for k in grouped_all_trials_df["trial_outcome"].unique()}

In [ ]:
outcome_to_style

In [ ]:
num_lines = 3

In [ ]:
# Generate colors from the "Blues" colormap
losing_colors = cm.Oranges(np.linspace(0.5, 1, num_lines))
# Generate colors from the "Blues" colormap
winning_colors = cm.Blues(np.linspace(0.5, 1, num_lines))
# Generate colors from the "Blues" colormap
rewarded_colors = cm.Greens(np.linspace(0.5, 1, num_lines))
# Generate colors from the "Blues" colormap
omission_colors = cm.Reds(np.linspace(0.5, 1, num_lines))

In [ ]:
outcome_to_color = {'lose': losing_colors[2],
 'lose_baseline': losing_colors[0],
 'omission': omission_colors[2],
 'omission_baseline': omission_colors[0],
 'rewarded': rewarded_colors[2],
 'rewarded_baseline': rewarded_colors[0],
 'win': winning_colors[2],
 'win_baseline': winning_colors[0]}

In [ ]:
outcome_to_color = {'lose': "orange",
 'lose_baseline': losing_colors[0],
 'omission': "red",
 'omission_baseline': "hotpink",
 'rewarded': "green",
 'rewarded_baseline': rewarded_colors[0],
 'win': "blue",
 'win_baseline': winning_colors[0]}

In [ ]:
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    
    for subj in region_df["current_subject"].unique():
        subj_df = region_df[region_df["current_subject"] == subj]
    
        fig, ax = plt.subplots()
        plt.title("Z-Scored LFP Power in {} for subject {}".format(region, subj))
        plt.xlabel("Frequency")
        plt.ylabel("Power")
        plt.xlim(0, 12)
        plt.ylim(0, 0.15)
        # plt.yscale("log")
    
        for outcome in subj_df["trial_outcome"].unique():
            outcome_df = subj_df[subj_df["trial_outcome"] == outcome]
            for index, row in outcome_df.iterrows():
                try:
                    # print(row["mean_power"])
                    ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], label="{}".format(outcome), color=outcome_to_color[outcome], linestyle=outcome_to_style[outcome], linewidth=3)
                    # ax.lines[0].set_linestyle(outcome_to_style[outcome])
                    plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2, color=outcome_to_color[outcome])
                except Exception as e: 
                    print(e)
                    continue
        ax.legend(bbox_to_anchor=(1.3, 0.9), loc='upper right')
        plt.tight_layout()
        plt.savefig("./proc/z_scored_all_conditions_{}_subj_{}.png".format(region, subj))
        plt.show()
    # # Removing duplicate labels in the legend
    # handles, labels = plt.gca().get_legend_handles_labels()
    # unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    # plt.legend(*zip(*unique))

# Drawing plots all together

In [ ]:
grouped_all_trials_df = exploded_channel_map_and_all_trials_df.groupby(['brain_region', 'trial_outcome']).agg({'power': lambda x: np.vstack(x.tolist())}).reset_index()

In [ ]:
grouped_all_trials_df["mean_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanmean(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["std_power"] = grouped_all_trials_df["power"].apply(lambda x: np.nanstd(np.vstack(x), axis=0))

In [ ]:
grouped_all_trials_df["n_trials"] = grouped_all_trials_df["power"].apply(lambda x: np.sum(~np.isnan(x), axis=0))

In [ ]:
grouped_all_trials_df["sem_power"] = grouped_all_trials_df.apply(lambda x: x["std_power"] / np.sqrt(x["n_trials"]), axis=1)

In [ ]:
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    


    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(0, 12)

    # plt.ylim(0, 0.15)
    # plt.yscale("log")

    for outcome in sorted(region_df["trial_outcome"].unique())[::-1]:
        outcome_df = region_df[region_df["trial_outcome"] == outcome]
        for index, row in outcome_df.iterrows():
            try:
                # print(row["mean_power"])
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], label="{}".format(outcome), color=outcome_to_color[outcome], linestyle=outcome_to_style[outcome], linewidth=3)
                # ax.lines[0].set_linestyle(outcome_to_style[outcome])
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2, color=outcome_to_color[outcome])
            except Exception as e: 
                print(e)
                continue
    ax.legend(bbox_to_anchor=(1.3, 0.9), loc='upper right')
    plt.tight_layout()
    plt.savefig("./proc/z_scored_all_conditions_all_subj_{}_0_to_12hz.png".format(region, subj))
    plt.show()
    # # Removing duplicate labels in the legend
    # handles, labels = plt.gca().get_legend_handles_labels()
    # unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    # plt.legend(*zip(*unique))

In [ ]:
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    


    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(0, 12)

    plt.ylim(0, 0.15)
    # plt.yscale("log")

    for outcome in sorted(region_df["trial_outcome"].unique())[::-1]:
        outcome_df = region_df[region_df["trial_outcome"] == outcome]
        for index, row in outcome_df.iterrows():
            try:
                # print(row["mean_power"])
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], label="{}".format(outcome), color=outcome_to_color[outcome], linestyle=outcome_to_style[outcome], linewidth=3)
                # ax.lines[0].set_linestyle(outcome_to_style[outcome])
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2, color=outcome_to_color[outcome])
            except Exception as e: 
                print(e)
                continue
    ax.legend(bbox_to_anchor=(1.3, 0.9), loc='upper right')
    plt.tight_layout()
    plt.savefig("./proc/z_scored_all_conditions_all_subj_{}_0_to_12hz_with_ylim.png".format(region, subj))
    plt.show()
    # # Removing duplicate labels in the legend
    # handles, labels = plt.gca().get_legend_handles_labels()
    # unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    # plt.legend(*zip(*unique))

In [ ]:
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    


    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(12, 30)

    plt.ylim(0, 0.01)
    # plt.yscale("log")

    for outcome in sorted(region_df["trial_outcome"].unique())[::-1]:
        outcome_df = region_df[region_df["trial_outcome"] == outcome]
        for index, row in outcome_df.iterrows():
            try:
                # print(row["mean_power"])
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], label="{}".format(outcome), color=outcome_to_color[outcome], linestyle=outcome_to_style[outcome], linewidth=3)
                # ax.lines[0].set_linestyle(outcome_to_style[outcome])
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2, color=outcome_to_color[outcome])
            except Exception as e: 
                print(e)
                continue
    ax.legend(bbox_to_anchor=(1.3, 0.9), loc='upper right')
    plt.tight_layout()
    plt.savefig("./proc/z_scored_all_conditions_all_subj_{}_12_to_30hz.png".format(region, subj))
    plt.show()
    # # Removing duplicate labels in the legend
    # handles, labels = plt.gca().get_legend_handles_labels()
    # unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    # plt.legend(*zip(*unique))

In [ ]:
for region in grouped_all_trials_df["brain_region"].unique():
    region_df = grouped_all_trials_df[grouped_all_trials_df["brain_region"] == region]
    


    fig, ax = plt.subplots()
    plt.title("Z-Scored LFP Power in {}".format(region))
    plt.xlabel("Frequency")
    plt.ylabel("Power")
    plt.xlim(30, 90)

    plt.ylim(0, 0.002)
    # plt.yscale("log")

    for outcome in sorted(region_df["trial_outcome"].unique())[::-1]:
        outcome_df = region_df[region_df["trial_outcome"] == outcome]
        for index, row in outcome_df.iterrows():
            try:
                # print(row["mean_power"])
                ax = sns.lineplot(x=exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], y=row["mean_power"], label="{}".format(outcome), color=outcome_to_color[outcome], linestyle=outcome_to_style[outcome], linewidth=3)
                # ax.lines[0].set_linestyle(outcome_to_style[outcome])
                plt.fill_between(exploded_channel_map_and_all_trials_df["frequencies"].iloc[0], row["mean_power"] - row["sem_power"], row["mean_power"] + row["sem_power"], alpha=0.2, color=outcome_to_color[outcome])
            except Exception as e: 
                print(e)
                continue
    ax.legend(bbox_to_anchor=(1.3, 0.9), loc='upper right')
    plt.tight_layout()
    plt.savefig("./proc/z_scored_all_conditions_all_subj_{}_30_to_90hz.png".format(region, subj))
    plt.show()
    # # Removing duplicate labels in the legend
    # handles, labels = plt.gca().get_legend_handles_labels()
    # unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    # plt.legend(*zip(*unique))